In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

#This line will hide code by default when the notebook is exported as HTML
#di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

import hail as hl
hl.init(tmp_dir='/net/scratch/people/plggosborcz', spark_conf={'spark.driver.memory': '10G', 'spark.executor.memory': '10G'}, default_reference='GRCh38') 

Running on Apache Spark version 2.4.5
SparkUI available at http://p0709.prometheus:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.62-84fa81b9ea3d
LOGGING: writing to /net/archive/groups/plggneuromol/imdik-zekanowski-gts/preprocessing/burden-and-family/python-scripts/hail-20211028-1130-0.2.62-84fa81b9ea3d.log


In [2]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()


import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain

from bokeh.plotting import output_notebook, show, figure
from bokeh.palettes import viridis

output_notebook() 

Loading BokehJS ...

Loading BokehJS ...

In [3]:
rpmk = hl.import_bed('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/external-data/repeatmasker_all', reference_genome='GRCh38', skip_invalid_intervals=True)

2021-10-27 17:06:39 Hail: INFO: Reading table without type imputation
  Loading field 'f0' as type str (user-supplied)
  Loading field 'f1' as type int32 (user-supplied)
  Loading field 'f2' as type int32 (user-supplied)


In [8]:
new_start = hl.if_else(((rpmk.interval.start.position == 1) | (rpmk.interval.start.position == 2)),
                       hl.locus(rpmk.interval.start.contig, rpmk.interval.start.position, reference_genome='GRCh38'),
                       hl.locus(rpmk.interval.start.contig, rpmk.interval.start.position-2, reference_genome='GRCh38'))      

new_end = hl.if_else(((rpmk.interval.end.position == hl.contig_length(rpmk.interval.end.contig, reference_genome='GRCh38')) | (rpmk.interval.end.position+1 == hl.contig_length(rpmk.interval.end.contig, reference_genome='GRCh38'))),
                       hl.locus(rpmk.interval.end.contig, rpmk.interval.end.position, reference_genome='GRCh38'),
                       hl.locus(rpmk.interval.end.contig, rpmk.interval.end.position+2, reference_genome='GRCh38'))
                     
                     

In [12]:
rpmk = rpmk.annotate(interval_extended = hl.interval(new_start, new_end))

In [14]:
rpmk.write('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/external-data/repeatmasker-extended.ht')

2021-10-27 17:21:56 Hail: INFO: Ordering unsorted dataset with network shuffle
2021-10-27 17:23:08 Hail: INFO: wrote table with 5520017 rows in 1 partition to /net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/external-data/repeatmasker-extended.ht
    Total size: 169.76 MiB
    * Rows: 169.76 MiB
    * Globals: 11.00 B
    * Smallest partition: 5520017 rows (169.76 MiB)
    * Largest partition:  5520017 rows (169.76 MiB)


In [18]:
rpmk = hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/external-data/repeatmasker-extended.ht')

In [19]:
rpmk = rpmk.key_by(rpmk.interval_extended)

In [20]:
rpmk.checkpoint('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/external-data/repeatmasker-extended-keyed.ht')

2021-10-27 17:24:22 Hail: INFO: Coerced sorted dataset
2021-10-27 17:24:36 Hail: INFO: wrote table with 5520017 rows in 1 partition to /net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/external-data/repeatmasker-extended-keyed.ht
    Total size: 186.13 MiB
    * Rows: 186.13 MiB
    * Globals: 11.00 B
    * Smallest partition: 5520017 rows (186.13 MiB)
    * Largest partition:  5520017 rows (186.13 MiB)


In [3]:
cov = hl.import_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/external-data/gnomad/gnomad.genomes.r3.0.coverage.summary.tsv.bgz')

2021-10-27 18:46:44 Hail: INFO: Reading table without type imputation
  Loading field 'locus' as type str (not specified)
  Loading field 'mean' as type str (not specified)
  Loading field 'median' as type str (not specified)
  Loading field 'over_1' as type str (not specified)
  Loading field 'over_5' as type str (not specified)
  Loading field 'over_10' as type str (not specified)
  Loading field 'over_15' as type str (not specified)
  Loading field 'over_20' as type str (not specified)
  Loading field 'over_25' as type str (not specified)
  Loading field 'over_30' as type str (not specified)
  Loading field 'over_50' as type str (not specified)
  Loading field 'over_100' as type str (not specified)


In [6]:
cov = cov.filter(hl.float(cov.over_1) > 0.9)

In [ ]:
cov.checkpoint('/net/scratch/people/plggosborcz/temp-mts/cov-temp.ht')

In [3]:
cov = hl.read_table('/net/scratch/people/plggosborcz/temp-mts/cov-temp.ht')

In [7]:
cov = cov.select(cov.locus)

In [8]:
cov.checkpoint('/net/scratch/people/plggosborcz/temp-mts/cov-loci-90-over-1.ht')

2021-10-28 11:44:35 Hail: INFO: wrote table with 2781559281 rows in 532 partitions to /net/scratch/people/plggosborcz/temp-mts/cov-loci-90-over-1.ht
    Total size: 21.08 GiB
    * Rows: 21.08 GiB
    * Globals: 11.00 B
    * Smallest partition: 118 rows (779.00 B)
    * Largest partition:  5587485 rows (43.40 MiB)


In [9]:
cov = cov.key_by(cov.locus)

In [11]:
cov = hl.read_table('/net/scratch/people/plggosborcz/temp-mts/cov-loci-90-over-1.ht')

In [12]:
cov.show()

""
locus
str
"""chr1:10028"""
"""chr1:10029"""
"""chr1:10030"""
"""chr1:10034"""
"""chr1:10035"""
"""chr1:10036"""
"""chr1:10040"""
"""chr1:10041"""


In [16]:
cov = cov.annotate(parsed_locus = hl.parse_locus(cov.locus, reference_genome='GRCh38'))

In [17]:
cov.write('/net/scratch/people/plggosborcz/temp-mts/gnomad-cov.ht')

2021-10-28 12:44:04 Hail: INFO: wrote table with 2781559281 rows in 532 partitions to /net/scratch/people/plggosborcz/temp-mts/gnomad-cov.ht
    Total size: 29.10 GiB
    * Rows: 29.10 GiB
    * Globals: 11.00 B
    * Smallest partition: 118 rows (1.11 KiB)
    * Largest partition:  5575404 rows (62.03 MiB)


In [23]:
cov = cov.key_by(cov.parsed_locus)

In [24]:
cov.show()

,
locus,parsed_locus
str,locus<GRCh38>
"""chr1:10028""",chr1:10028
"""chr1:10029""",chr1:10029
"""chr1:10030""",chr1:10030
"""chr1:10034""",chr1:10034
"""chr1:10035""",chr1:10035
"""chr1:10036""",chr1:10036
"""chr1:10040""",chr1:10040
"""chr1:10041""",chr1:10041


In [25]:
cov.write('/net/scratch/people/plggosborcz/temp-mts/gnomad-cov-keyed.ht')

2021-10-28 13:08:15 Hail: INFO: Coerced sorted dataset
2021-10-28 13:28:09 Hail: INFO: wrote table with 2781559281 rows in 532 partitions to /net/scratch/people/plggosborcz/temp-mts/gnomad-cov-keyed.ht
    Total size: 38.98 GiB
    * Rows: 38.98 GiB
    * Globals: 11.00 B
    * Smallest partition: 118 rows (1.84 KiB)
    * Largest partition:  5587485 rows (83.19 MiB)


#script to save:

import hail as hl
import sys                                                                                      
part = sys.argv[1]
hl.init(tmp_dir='/net/scratch/people/plggosborcz', spark_conf={'spark.driver.memory': '10G', 'spark.executor.memory': '10G'}, default_reference='GRCh38') 

rpmk = hl.read_table('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/external-data/repeatmasker-extended-keyed.ht')
cov = hl.read_table('/net/scratch/people/plggosborcz/temp-mts/gnomad-cov-keyed.ht')

hl.import_vcf('/net/archive/groups/plggneuromol/imdik-zekanowski-gts/data/joint-with-sportsmen/vcf-parts/'+str(part)+'-part.vcf.gz',
    reference_genome='GRCh38',
    force_bgz = True,
    array_elements_required = False).write('/net/scratch/people/plggosborcz/temp-mts/'+str(part)+'-part.mt')

mt = hl.read_matrix_table('/net/scratch/people/plggosborcz/temp-mts/'+str(part)+'-part.mt')
mt = mt.filter_rows(hl.is_defined(rpmk[mt.locus]), keep = False)

mt.checkpoint('/net/scratch/people/plggosborcz/temp-mts/'+str(part)+'-part-rpmk.mt')
mt = mt.filter_rows(hl.is_defined(cov[mt.locus]), keep = True)
mt.checkpoint('/net/scratch/people/plggosborcz/temp-mts/'+str(part)+'-part-cov.mt')


mt = mt.annotate_cols(group = hl.if_else(mt.s.contains('B'), 'sport', 'GTS'))
mt = mt.annotate_rows(dp_qc = hl.agg.group_by(mt.group, hl.agg.stats(mt.DP)),
                     gq_qc = hl.agg.group_by(mt.group, hl.agg.stats(mt.GQ)),
                     hwe = hl.agg.group_by(mt.group, hl.agg.hardy_weinberg_test(mt.GT)))

mt = mt.annotate_rows(n_below_dp_3 = hl.agg.group_by(mt.group, hl.agg.count_where(mt.DP < 3)),
                      n_below_gq_30 = hl.agg.group_by(mt.group, hl.agg.count_where(mt.GQ <30)))

mt.checkpoint('/net/scratch/people/plggosborcz/temp-mts/'+str(part)+'-qc.mt')

mt = mt.filter_rows((mt.dp_qc['sport'].mean > 5) &
                    (mt.dp_qc['GTS'].mean > 5) &
                    (mt.gq_qc['sport'].mean > 50) &
                    (mt.gq_qc['GTS'].mean > 50) &
                    (mt.hwe['sport'].p_value > 0.05) &
                    (mt.hwe['GTS'].p_value > 0.05) &
                    (mt.n_below_dp_3['sport'] < 3) &
                    (mt.n_below_gq_30['sport'] < 30) &
                    (mt.n_below_dp_3['GTS'] < 3) &
                    (mt.n_below_gq_30['GTS'] <30))

mt.checkpoint('/net/scratch/people/plggosborcz/temp-mts/'+str(part)+'-filtered.mt')